# Deep RL Bootcamp 2017

![Reinforcement Lear](../assets/rl_landscape.png)

# Lecture 1: Motivation + Overview + Solution Methods
## Pieter Abbeel

For now, image small discrete state-action space as they are simpler to get but in the future these will be large/ continuous spaces

#### Value Iteration
Based around the value function V*
V* is the sum of discounted rewards when starting from state s and acting optimally

Theorem: Value iteration converges. At convergence, we have found the optimal value function V* for the discouted infinite horizon problem, which satisfies the bellman equation. 

$V*(s) = max_a \Sigma_{s'} P(s'|s, a)[R(s, a, s') + \gamma V*(s')]$

#### Convergence and Contractions
If you do an update your value function from k to k+1 there is contraction
(He doesn't go into this but leaves you to understand it later, to prove something about the algorithm)

#### Discount Factor and Noise
A good demo around minute 30 showing how combinations of discount factor and noise change how an angent will act. A **discount factor** close to 1 means you care about the distant future and a small discount factor like .1 means you favor quick rewards because  long travels will be discount aggressively. 

#### Q-Values
A downside of Value iteration is the need to keep track of both a policy gradient and a value gradient. A Q-value combines these two notions by satisfying the Bellman Equation. A q-value encodes implicitly the optimal policy. There are situations we will get back to where we can use Q and V won't work and vice versa. 

#### Policy Evaluation
We take the same equation for value iteration but we assume a situation we instead of an action we take the policy $\pi(s)$. To iterate over a policy we evaluate a policy pi-k by starting with a current policy and then taking a single step ahead and looking to improve out policy. We find the best action according to one-step look-ahead. When the policy converges we have an optimal policy and an optimal value function. 

**A rewording I like from Lecture 2 about policy evaluation**
You start with a policy and you collect samples according to this policy, you use these samples to update the policy. 

These are methods to solve small MDPs. 

# Lecture 2: Sampling-based Approximations and Function Fitting
## Rocky Duan

The Bellman Equation when solved gives us Q*. 

#### Q-Value Iteration
Consider your new sample estimate:

$target(s') = R(s, a, s') + \gamma max_{a'}Q_k(s', a')$

Incorporate the new estimate into a running average:

$Q_{k+1}(s,a) \leftarrow (1-\alpha)Q_k(s,a) + \alpha[target(s')]$

Starting with a small alpha. 

#### Sample Actions
Choosing radomly versus greedily
Epsilon-greedy- Choose randomly epsilon, and greedily 1-epsilon. 

He makes a small deviation to talk about **model-based rl** where the agent attempts to understand the transition function and to build a model of the environment and then apply a series of **planning** based solutions to that model. 

#### Q-Learning Properties

Off-policy learning
You have to explore enough
You have to make the learning rate small enough but not decrease it too quickly

One way of dealing with **learning rate scheduling** is to have learning rate inversely proportional to steps. 

#### Sample-Based Approximation

I really dont understand and wish I understood this section, it was quick but seemed to show approaches that were and were not amenable to a sample based approximation. 

**TD, Temporal Distance** is so named because it is the difference between time steps like in policy iteration evaluation where the loss is computed between two steps. 

# Lecture 3: Deep Q-Networks
## Vlad Mnih

In Q learning we define a target for the current q value which depends the reward you got and the the q value of the best action in the next state. Q learning is iterative regression, where we fit a q value to the targets we computed. We optimize a squared loss function on these targets. Here is the formula/slide notes on Q-learning update:

![qlearn_update](../assets/qlearn_update.png)

This lecture is about making q-learning work with neural netowrks, which requires some tricks. If you reach a terminal state then your target is 
your reward and we are calculating these targets at every state and if you don't account for the different situations your model won't converge. The update, in the tabular case, is the (1-alpha)* Q + (alpha* the target value)

Neural networks generalize between states, so when you update the q-value for some state and action, because the neural net generalizes it will change the other states and actions and you end up with a nonstationary aspect. The other trick is about dealing with is correlation in rollouts. 

The goal with DQN was to make neural nets work like they did in supervised learning. Two main ideas for stabalizing Q-learning. Apply Q-updates on batches of past experience, better data efficiecy and make the data distribution more stationary. Use an older set of weights to compute the targets (target network) keeps the target function from changing too quickly. 

The size of the memory buffer matters and is its own hyperparameter- when google runs experiments they use a buffer of at least a million. 

Dont forget in the training to deal with terminal and non-terminal states where terminal states only equal the reward. 

#### DQN Implementation Details
They don't use squared loss they use Huber loss in the on Bellman error, and RMSProp. 

In reinforcement learning optimization matters a lot, it isn't just about fitting a dataset, the optimizer determines what your agent will do and what they will see. 

How Deepmind **anneals the learning rate**. The learning rate is high during the initial learning phase and then it anneals linearly to .1 or .05 where it stays for the duraiton of learning. It helps to have a phase with a lot of exploration at the begining. 

There are games where it cannot learn just through random exploration, like Montezuma. In this case we can sample from two buffers, one with the game generatd data and a buffer made from human play information. 

Preprocessing included down sampling and greyscaling the game frames.

#### Double DQN
There is an overestimation bias when using a function approximator to represent Q.
Double DQN maintains two sets of weights theta and theta', so reduce bias by using:
1. $\theta$ for selecting the best action
2. $\theta'$ for evaluating the best action

Double DQN is more stable and uses lower estimations of Q more often

Use the **online network** to select the best action, and use the **target network** to get the value estimate.

#### Prioritized Experience Replay
Ideally when sampling from our buffer we don't want a uniform smaple, we want to sample from the state-space where we are most wrong. You can sample proportional to the Bellman erroor 

#### Dueling DQN
We know both Q and V. There is another quantity that is called the **advantage** which is the difference between the q-value and the state-value. So this essentially tells you how much worse an action is than the best action in the state. So we change the fully-connected output layer to two seperate channels that output the value, and the advantage- if we sum these up we get an estimate of the q-value. 

#### Noisey Nets for Exploration 
You can go way beyound epsilon-greedy exploration and one good way is to just add noise to the paramters of the neural networks. This is from recent papers from 2017. 

# Lecture 4A: Policy Gradients
## Pieter Abbeel

Policy gradients differ from dynamic programming methods which look to satisfy a set of self-consistent equations, like q-learning. 

We replace q-values with a policy, pi, that is a stochastic that ia a distribution of our actions given a state that maximizes the expected sum of rewards. Action, unlike in dynamic programming will be denoted with $u_t$. 

Theta parameterizes the policy. 
Gamma, used for discouting will equal 1 in this lecture.

Pi can be simpler than Q or V

A Policy Gradient is less sample efficient than dynamic programming 

To find the policy we use:
#### Likelihood Ratio Policy Gradient
We let tau denote a state-action sequence, the first part of the below equation where Utility U is denoted as the sum of state, action pairs. Reward is the entire trajectory, it the sum of individual rewards along the trajectory. 

$U(\theta)=E[\Sigma_{t=0}R(s_t, u_t);\pi_{\theta}] = \Sigma_{\tau}P(\tau;\theta)R(\tau)$

If you were going to try something first- as just the first thing, policy gradient is the first thing you should try. 

The max of utility parameterized by theta is the max of this expectation:
 
$maxU(\theta) = max_{\theta}\Sigma_{\tau}P(\tau;\theta)R(\theta)$
 
To compute an estimate of the policy gradient, we have i indexes sample paths and m sample paths 

$\Delta_{\theta}(\theta) \approx gradient =\frac{1}{m}{\Sigma}^{m}_{i=1}\Delta_{\theta}logP(\tau^i;\theta)R(\tau^i)$

This gradient function is valid even if R is discontinuous or unknown and sample paths, is a discrete set. 

PPO (proximal policy optimization) is the state of the art and might be your first choice if you just throw an algorithm at something. 

#### Importance Sampling

It when you compute an expectation under some distribution, namely under policy $\pi_\theta$, you want to know what the expectation is but your samples, your trajectoies come from some other policy $\pi_{\theta0}$. So you sample from Pi theta O which give you trajectories- and you use those trajectories to compute a new theta, and you correct with this ratio

$U(\theta) = E_{\tau ~ \theta_{old}}[\frac{P(\tau|\theta)}{P(\tau|\theta_{old})}R(\tau)]$

This ratio performs the correction. This is a way to compute the expectation we care about but shows it as a way of computing it from samples from an old policy rather than the current policy. 

Derivation from Importance Sampling:
We can use this when we are curious about a new policy so we use an old policy with a finite perturbation to find how good the new policy is going to be. 

$\Delta_\theta U(\theta)|_{\theta=\theta_{old}}=E_{\tau \theta_{old}}[\Delta_\theta log P(\tau|\theta)|_{\theta old}R(\tau)]$

#### Likelihood Ratio Gradient: Baseline

Imagine a scenario where we have several rollouts (sample paths of policy), where we have mostly negative rollouts. We want to increase probability of paths that are better than average and decrease the probability of those that are less than average then we can simply watch our rollouts, compute the average, and subtract that from our reward from our rollout.

As shown in Williams '92, we do not bias our gradient, we simply reduce variance. 

#### Likelihood Ratio and Temporal Structure

Removing terms that don't depend on current action can lower variance:

$\frac{1}{m}\Sigma^{m}_{i=1}\Sigma^{H-1}_{t=0}\Delta_\theta log \pi_\theta(u^{i}_{t}|s^{i}_{t})(\Sigma^{H-1}_{k=t}R(u^{i}_{k},s^{i}_{k})-b(s^{i}_{t}))$

We look only at rewards from actions after our current action. 

#### Optimal Constant Baseline
Rollouts that are more likely contribute to your average than rollouts that are less likely and can be added as a simple improvement to the baseline- even though Abbeel says he doesn't think anyone uses this. 

There are several baseline choices, with formulas on the slides
1. Constant baseline
2. Optimal constant baseline
3. Time-dependent baseline
4. State-dependent expected return
    - Increase logprb of action proportinally to how much its returns are better than the expected return under the current policy
    
To use state-dependent expected return we have to estimate $V^{\pi}$

When we have a transition model we can run policy evaluation but here we don't have a transition model so we need to estimate it from samples. We will have a seperate value function, parameterized by $\phi$. We want this netowrk, given a state, to predict whatever rewards we got from being in this state.  We can use supervised learning, input is state, we approximate a function for the reward and we now have a network for estimating a value function.

There are other ways of estimating $V^{\pi}$, we can use the bellman equation for V-pi, which is a monte carlo estimate of V-pi. We can also use a TD estimate for V-pi, that uses target values and iterates to get a better estimation.

The simplest baseline is not very sample efficient. 

#### Vanilla Policy Gradient

1. Initialize policy parameter theta, baseline b
2. for iteration do 
    1. Collect a set of trajectories by executing the current policy
    2. At each timestep in eahc trajectory, compute the return R_t and the advantage estimate, $A_t = R_t-b(s_t)$
    3. Re-fit the baseline, by minimizing the distance squared between baseline and reward, summed over all trajectories and timesteps
    4. Update the policy, using a policy gradient estimate G (I omiited the formula)
   
**Recall our Likelihood ratio PG Estimator**
Our Sum or rewards term is the same as the estimation of Q from a single roll-out
On average we will get the Q value, but if we just use the rollout we are going to get high variance. We can reduce the variance with discounting, even better with function approximation. 

The expected value is the expeceted value right now and the expected value of the next state, and some amount of future states. A3C uses this exact same idea, with a lookahead of k=5. You use the sum of the 5 future rewards followed by your value function estimate. 

There is also **generalized advantage estimation** where there is exponential averaging of all posisble k values you could pick, called lambda based weighting. Related to TD(lambda)/eligibility traces in Sutton and Barto.

Once you have a policy gradient where you estimate a value function for your baseline you officially have an **actor-critic** 

There are many orthoganol design decisions: collect rollouts and acumulated rewards, use monte carlo vs AC3- cuttout after a few steps or Generlaized Advantage Estimation.

Ac3 is a much better algorithm for Beamrider, Breakout, Pong, Q'bert, and Space Invaders. 

# Lecture 4b: Policy Gradients Revisited
## Andrej Karpathy

In reinforcement learning we have no ground truth:

$y_i \sim p(-|x_i)$

Where Y is distributed according to the distribution probability of $p(-|x_i)$

Once we colllect a batch of rollouts:

$\Sigma_i A_i * log p(y_i|x_i)$

We call A_i the advantage, it's a number like -1.0 or 1.0 based on how this action will eventullay turned out. 

#### Advantage Function
It is a scalar that is used to modulate with a negaitve or positive if we are likely to see that action in the future. In this context the advantage is like the reward function. 

From Karpathy: "The advantage is some kind of thing- its a scalar that tells you how much you want to encourage or discourage the action you happen to take"

There are some principle ways you choose the advatage, you can take the empirical return and subtract the baseline- there are multiple ways of using advatage. There is something at around 17:20 Karpathy is talking about advantage function as a Gaussian distribution that we sample from increase/decrease the probability of that transition state- I think.

During a question at the tail end he addresses as key issue in the discounted reward. 
You can use the **discounted reward as the advantage**, and at first I did, but the discounted reward can move all over the place based on the reward so I wanted to make it zero mean and scale it to a more balanced place. I coudn't see exactly what in the code he did to change discounted reward to the advantage function but this confirms that the relationship between the advantage and the discounted reward are similar. 

# Lecture 5: Natural Policy Gradients, TRPO, PPO
## John Shulman

These and the following talks by Shulman are more advanced optimization methods to use with policy gradients. 

Vanilla Policy Gradient is essentially getting a policy estimate and plugging it into stichastic gradient descent. 

Actor critic methods use the value function to get a lower variance advantage estimates. But this talk isn't about getting better advantage esitmates its about changing the underlying optimization algorithm. 

#### LImitations to Vanilla Policy Gradient
1. Hard to choose stepsizes. Input data is nonstationary due to changing policy: observation and reward distribution change
2. Bad step is more damaging than in supervised laerning, sinve it affects visitation distribution. You end up with a negaitve feedback loop becuase you are iterated over policies, a bad policy changes the data you collect for the next batch update and you end up with a collapse in performance you can't recover from. 
3. Sample efficiency. Only one gradient step per environment sample, dependent on scaling of coordinates. 

**Much of modern ML is reducing learning to numerical optimization, in supervised learning this is about minimizing training error. Learning is about making some kind of prediction on data you haven't seen before**

#### How to compute the best data with the policy we have

1. Q-learning- In principle can include all transitions seen so far, however, we're optiizing the wrong objective
2. Policy gradients- Yes stochastic gradients, but no optmization problem.
3. This lecture- Write down an optimization problem that allows you to do a small update to policy pi based on data smapled from pi (on-policy data)

You can any set of transitions from your **dynamics model** and can fit a q-function using them. This comes at the cost of optmizing the wrong objective- you obtimize Bellman error but what you care about is that the policy does what you want. 

Here we are going to solve an optimization problem with the data that we collect with our policy. 

#### Trust Region Policy Optimization (TRPO)

From operations research, we have some trust region where we can move around and have a policy with low error. We only move away from our starting point by some euclidean distance theta, we can also use KL divergence. A note from the lecture in regards to using wassterstein or other distance metrics- there's not too much to say about distances between different Gaussian distributions. 

How we use trust region policy optimization
1. Run policy for T timesteps or N trajectories
2. Estimate advantage function for each timestep 
3. Then approx solve our constrianed optimization problem 
4. Update usign conjugate gradient

This is similar to natural policy gradient, natural actor-critic, and REPS

#### Connection between Trust Region Problem and Other Things

Linear-quadratic approximation + penalty => natural gradient
No constrain => policy iteration 
Euclidean penalty instead of KL => vanilla policy gradient. 

No constraint is like policy iteration because in policy iteration you're pciking a deterministic policy each iteration in which you chose an action that had the highest advantage according to the previous policy. 

In **KFAC** we are doing something similar to TRPO but we are doin a more compex update step. 